In [116]:
!{sys.executable} -m pip install nltk
#Required when running from a newly created spark docker container

In [118]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#required for nltk functionality

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [105]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from pyspark.ml.feature import CountVectorizer
import sys
import string
from collections import defaultdict

In [4]:
import sys

In [5]:
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()

In [6]:
sc = spark.sparkContext
spk = SparkSession.builder.master('local').getOrCreate()

In [15]:
df = spark.read.csv("/home/jovyan/work/galvanize/NLP-with-Amazon-reviews/data/gaming_reviews.tsv", sep="\t", header=True, inferSchema=True)

In [17]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
df_y = df.select('product_title', 'star_rating', 'review_body').withColumn('good_product', df['star_rating'] > 3)

In [ ]:
df_y.select('product_title').take(20);

In [ ]:
df_y.createTempView('products')

In [ ]:
spk.sql("SELECT review_body FROM products WHERE product_title = 'Super Mario Galaxy 2'").take(50);

In [57]:
df_ratings = df.select('star_rating', 'review_body').limit(1000)

In [ ]:
remove_punct(df_ratings.select('review_body').head()[0]).split()

In [132]:
type(df_ratings)

pyspark.sql.dataframe.DataFrame

In [139]:
word_list = process_str(df_ratings.select('review_body').head()[0])

['Used',
 'Elite',
 'Dangerous',
 'mac',
 'amazing',
 'joystick',
 'I',
 'especially',
 'love',
 'twist',
 'stick',
 'different',
 'movement',
 'bindings',
 'well',
 'move',
 'normal',
 'way']

In [167]:
process_str(df_ratings.select('review_body').head()[0])

['Used',
 'Elite',
 'Dangerous',
 'mac',
 'amazing',
 'joystick',
 'I',
 'especially',
 'love',
 'twist',
 'stick',
 'different',
 'movement',
 'bindings',
 'well',
 'move',
 'normal',
 'way']

In [142]:
stop_words = set(stopwords.words('english'))

In [144]:
def process_str(row):
    """Input: String
    Output: List of strings without punctuation
    
    Removes punctuation using functions from the strings library. Performs raw string operations in C via a lookup table for maximum performance."""
    word_list = row.translate(str.maketrans('', '', string.punctuation)).split(' ')
    return [x for x in word_list if x not in stop_words]

In [177]:
process = udf(lambda mylist: process_str(mylist), DataType: Array)

SyntaxError: invalid syntax (<ipython-input-177-127fd3430a68>, line 1)

In [169]:
df_new = df_ratings.withColumn('body_list', process(df_ratings['review_body']))
df_new = df_new.select('star_rating', 'body_list')

In [170]:
type(df_new)

pyspark.sql.dataframe.DataFrame

In [171]:
df_new.head()

Row(star_rating=5, body_list='[Used, Elite, Dangerous, mac, amazing, joystick, I, especially, love, twist, stick, different, movement, bindings, well, move, normal, way]')

In [114]:
word_dict = defaultdict(lambda: 0)

In [162]:
df_new.select('body_list').head()[0]

'[Used, Elite, Dangerous, mac, amazing, joystick, I, especially, love, twist, stick, different, movement, bindings, well, move, normal, way]'

In [158]:
cv = CountVectorizer(inputCol="body_list", outputCol="features")

In [159]:
cv.fit(df_new)

IllegalArgumentException: 'requirement failed: Column body_list must be of type equal to one of the following types: [array<string>, array<string>] but was actually of type string.'